In [4]:
# ✅ EcoVision-AI Pollution Detection - Model Training
# This version auto-fixes dataset paths and runs end-to-end training.

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # pyright: ignore[reportMissingImports]
from tensorflow.keras import layers, models # pyright: ignore[reportMissingImports]
from tensorflow.keras.callbacks import EarlyStopping # pyright: ignore[reportMissingImports]

# ----------------------------------------------------
# ✅ Step 1: Set up dataset paths (auto-fix for notebooks folder)
# ----------------------------------------------------
base_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "dataset"))
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "validation")

print("Training directory:", train_dir)
print("Validation directory:", val_dir)

# Check if folders exist
if not os.path.exists(train_dir) or not os.path.exists(val_dir):
    raise FileNotFoundError(f"❌ Dataset folders not found! Check if 'dataset/train' and 'dataset/validation' exist.\nCurrent base_dir: {base_dir}")

# ----------------------------------------------------
# ✅ Step 2: Image parameters
# ----------------------------------------------------
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# ----------------------------------------------------
# ✅ Step 3: Data Generators
# ----------------------------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("\n✅ Data generators ready. Classes found:", train_generator.class_indices)

# ----------------------------------------------------
# ✅ Step 4: Build CNN Model
# ----------------------------------------------------
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(*IMG_SIZE, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()

# ----------------------------------------------------
# ✅ Step 5: Compile the Model
# ----------------------------------------------------
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ----------------------------------------------------
# ✅ Step 6: Train the Model
# ----------------------------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

EPOCHS = 20

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stop]
)

# ----------------------------------------------------
# ✅ Step 7: Save the Model
# ----------------------------------------------------
model_save_path = os.path.join(os.getcwd(), "eco_model.h5")
model.save(model_save_path)
print(f"\n✅ Model saved successfully at: {model_save_path}")

# ----------------------------------------------------
# ✅ Step 8: Evaluate the Model
# ----------------------------------------------------
val_loss, val_acc = model.evaluate(val_generator)
print(f"\n📊 Validation Accuracy: {val_acc * 100:.2f}%")
print(f"📉 Validation Loss: {val_loss:.4f}")


Training directory: c:\Users\own\EcoVision-AI-Pollution-Detection\dataset\train
Validation directory: c:\Users\own\EcoVision-AI-Pollution-Detection\dataset\validation
Found 200 images belonging to 8 classes.
Found 200 images belonging to 8 classes.

✅ Data generators ready. Classes found: {'a_Good': 0, 'b_Moderate': 1, 'c_Unhealthy_for_Sensitive_Groups': 2, 'clean': 3, 'd_Unhealthy': 4, 'e_Very_Unhealthy': 5, 'f_Severe': 6, 'polluted': 7}


C:\Users\own\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,829,384 (18.42 MB)

 Trainable params: 4,829,384 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.1750 - loss: 2.0670 - val_accuracy: 0.1900 - val_loss: 1.8870
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 878ms/step - accuracy: 0.3000 - loss: 1.8335 - val_accuracy: 0.3100 - val_loss: 1.8348
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 900ms/step - accuracy: 0.2950 - loss: 1.7419 - val_accuracy: 0.2450 - val_loss: 1.7578
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 790ms/step - accuracy: 0.3650 - loss: 1.6376 - val_accuracy: 0.3300 - val_loss: 1.6807
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 854ms/step - accuracy: 0.4200 - loss: 1.5452 - val_accuracy: 0.2650 - val_loss: 1.6993
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 839ms/step - accuracy: 0.3650 - loss: 1.5520 - val_accuracy: 0.4000 - val_loss: 1.6205
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 902ms/step - accuracy: 0.4300 - loss: 1.4564 - val_accuracy: 0.3400 - val_loss: 1.6171
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 795ms/step - accuracy: 0.4050 - loss: 1.4300 - val_accuracy: 0.3900 - val_loss: 1


✅ Model saved successfully at: c:\Users\own\EcoVision-AI-Pollution-Detection\notebooks\eco_model.h5
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.3450 - loss: 1.5369

📊 Validation Accuracy: 34.50%
📉 Validation Loss: 1.5369
